In [ ]:
!pip install deep-translator
!pip install googletrans

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator
from googletrans import Translator
import re
import string
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import random
from tensorflow.keras import layers
from keras.layers import InputLayer, MaxPooling2D, Flatten, Dense, Conv2D, Dropout, BatchNormalization
import tensorflow as tf
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
% cd /content/gdrive/My Drive/Final Project

In [ ]:
train=pd.read_csv('training.csv')
test=pd.read_csv('test.csv')
valid=pd.read_csv('validation.csv')

In [ ]:
train['text'][0]

In [ ]:
idx=train.index

In [ ]:
test.info()

In [ ]:
valid.info()

In [ ]:
train.info()

In [ ]:
train2=train.sample(frac=1,random_state=913)
train2=train2.reset_index(drop=True)
train2

In [ ]:
#define pipeline translator
indo=GoogleTranslator(result='auto',target='id')

In [ ]:
idx=train2.index

In [ ]:
hasil=[]
for i in idx:
  indos=indo.translate(train2['text'][i])
  hasil.append(indos)

In [ ]:
train2['indo']=hasil

In [ ]:
train_indo=train2[['indo','label']]

In [ ]:
train_indo.sample(10)

# Data Preprocessing

In [ ]:
#lowering
train_indo['indo']=train_indo['indo'].apply(lambda x: x.lower())


for i in idx:
  indos=indo.translate(train2['text'][i])
  hasil.append(indos)

In [ ]:
idx2=train_indo.index
idx2

In [ ]:
#remove numbers
for i in idx2:
  train_indo['indo'][i] = re.sub(r'\d+', '', train_indo['indo'][i])

In [ ]:
#remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
train_indo['indo']= train_indo['indo'].apply(lambda x:remove_punctuation(x))

In [ ]:
stop_words=set(stopwords.words('indonesian'))

In [ ]:
#
train_indo['indo_token']=train_indo['indo']
for i in idx2:
  train_indo['indo_token'][i]=word_tokenize(train_indo['indo'][i])

In [ ]:
def remove_stopwords(text):
    output= [i for i in text if i not in stop_words]
    return output

In [ ]:
train_indo['indo_stop']= train_indo['indo_token'].apply(lambda x:remove_stopwords(x))

In [ ]:
train_indo=pd.read_csv('train_indo_token_stopwords.csv')

In [ ]:
train_indo.head()

In [ ]:
y=train_indo['label']

In [ ]:
train_set, temp_df = train_test_split(train_indo, test_size=0.15,random_state=123,stratify=y)

In [ ]:
y2=temp_df['label']

In [ ]:
test_set, valid_set = train_test_split(temp_df, test_size=0.5,random_state=123,stratify=y2)

In [ ]:
X_train, y_train = train_set.indo_stop, train_set.label
X_test, y_test = test_set.indo_stop, test_set.label
X_valid, y_valid = valid_set.indo_stop, valid_set.label

def prepare_target(y_train, y_test,y_valid):
  y_train_enc = to_categorical(y_train)
  y_test_enc = to_categorical(y_test)
  y_valid_enc = to_categorical(y_valid)
  return y_train_enc, y_test_enc,y_valid_enc

y_train_array, y_test_array,y_valid_array = prepare_target(y_train, y_test,y_valid)


In [ ]:
X_train[:10], y_train_array[:10]

In [ ]:
round(sum([len(i.split()) for i in train_set])/len(train_set))

In [ ]:
max_vocab_length = 4000
max_length = 20

text_vectorization = TextVectorization(max_tokens=max_vocab_length,
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None,
                                       output_mode='int',
                                       output_sequence_length=max_length)

In [ ]:
# vektorisasi teks
text_vectorization.adapt(X_train)

In [ ]:
# sms baru
sample = "aku merasa bahagia, pintar, cerdas, senang"
text_vectorization([sample])

In [ ]:
text_vectorization.get_vocabulary()[81]

In [ ]:
random_text = random.choice(train_set)
print(f"Teks Asli : {random_text}\n")
text_vectorization([random_text])

In [ ]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)

embedding

In [ ]:
random_text = random.choice(train_set)
print(f"Teks Asli : {random_text}\n")
text_vectorization([random_text])

sample_embedded = embedding(text_vectorization([random_text]))
sample_embedded.shape

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
x = Dropout(0.2)(x)
hidden_layer1=Dense(500,activation='relu',name='hl1',kernel_initializer=initializers.GlorotNormal(seed=123))(x)
batch1=BatchNormalization(axis=1)(hidden_layer1) 
hidden_layer2=Dense(250,activation='relu',name='hl2',kernel_initializer=initializers.GlorotNormal(seed=123))(batch1)
batch2=BatchNormalization(axis=1)(hidden_layer2) 
hidden_layer3=Dense(125,activation='relu',name='hl3',kernel_initializer=initializers.GlorotNormal(seed=123))(batch2)
batch3=BatchNormalization(axis=1)(hidden_layer3) 
hidden_layer4=Dense(63,activation='relu',name='hl4',kernel_initializer=initializers.GlorotNormal(seed=123))(batch3)
batch4=BatchNormalization(axis=1)(hidden_layer4) 
hidden_layer5=Dense(32,activation='relu',name='hl5',kernel_initializer=initializers.GlorotNormal(seed=123))(batch4)
batch5=BatchNormalization(axis=1)(hidden_layer5) 
outputs = layers.Dense(6, activation='softmax')(batch5)
model_1 = tf.keras.Model(inputs, outputs, name='model_nlp_fc')

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
x = layers.LSTM(64)(x)
x = Dropout(0.2)(x)
hidden_layer1=Dense(500,activation='relu',name='hl1',kernel_initializer=initializers.GlorotNormal(seed=123))(x)
batch1=BatchNormalization(axis=1)(hidden_layer1) 
hidden_layer2=Dense(250,activation='relu',name='hl2',kernel_initializer=initializers.GlorotNormal(seed=123))(batch1)
batch2=BatchNormalization(axis=1)(hidden_layer2) 
hidden_layer3=Dense(125,activation='relu',name='hl3',kernel_initializer=initializers.GlorotNormal(seed=123))(batch2)
batch3=BatchNormalization(axis=1)(hidden_layer3) 
hidden_layer4=Dense(63,activation='relu',name='hl4',kernel_initializer=initializers.GlorotNormal(seed=123))(batch3)
batch4=BatchNormalization(axis=1)(hidden_layer4) 
hidden_layer5=Dense(32,activation='relu',name='hl5',kernel_initializer=initializers.GlorotNormal(seed=123))(batch4)
batch5=BatchNormalization(axis=1)(hidden_layer5) 
outputs = layers.Dense(6, activation='softmax')(batch5)
model_1 = tf.keras.Model(inputs, outputs, name='model_nlp_fc')

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorization(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = Dropout(0.5)(x)
hidden_layer1=Dense(500,activation='relu',name='hl1',kernel_initializer=initializers.GlorotNormal(seed=123))(x)
batch1=BatchNormalization(axis=1)(hidden_layer1) 
hidden_layer2=Dense(250,activation='relu',name='hl2',kernel_initializer=initializers.GlorotNormal(seed=123))(batch1)
batch2=BatchNormalization(axis=1)(hidden_layer2) 
hidden_layer3=Dense(125,activation='relu',name='hl3',kernel_initializer=initializers.GlorotNormal(seed=123))(batch2)
batch3=BatchNormalization(axis=1)(hidden_layer3) 
hidden_layer4=Dense(63,activation='relu',name='hl4',kernel_initializer=initializers.GlorotNormal(seed=123))(batch3)
batch4=BatchNormalization(axis=1)(hidden_layer4) 
hidden_layer5=Dense(32,activation='relu',name='hl5',kernel_initializer=initializers.GlorotNormal(seed=123))(batch4)
batch5=BatchNormalization(axis=1)(hidden_layer5) 
outputs = layers.Dense(6, activation='softmax')(batch5)
model_1 = tf.keras.Model(inputs, outputs, name='model_nlp_fc')

In [ ]:
model_1.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1.summary()

In [ ]:
model_1_hist = model_1.fit(X_train,
                           y_train_array,
                           epochs=50,
                           validation_data=(X_valid, y_valid_array))

In [ ]:
tf.keras.utils.plot_model(model_1, show_shapes=True, rankdir="LR")

In [ ]:
results=model_1.predict(X_test)

In [ ]:
result_class=results.argmax(axis=1)

In [ ]:
print(classification_report(y_test, result_class))

In [ ]:
model_1_hist

In [ ]:
# visualize the chart

pd.DataFrame(list(zip(model_1_hist.history['accuracy'],model_1_hist.history['val_accuracy'])),
               columns =['accuracy', 'val_accuracy']).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()